In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("anarchia")

create_database('blp_dataset')
create_table('products_tmp_anarchia')

In [ ]:
p_img_hashes = {}
Image.MAX_IMAGE_PIXELS = None

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\index.php\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                content = page_content.find("table", attrs={"class": "itemInfo"})

                if content is not None:
                    desc_content = content.find("div", attrs={"class": "well"})

                    if desc_content is not None:
                        description = get_content(desc_content)

                    ## GET TITLE
                    title = ""
                    title_content = content.find("span", attrs={"class": "btn"})

                    if title_content is not None:
                        title = get_content(title_content)

                    ## GET SELLER
                    seller = ""
                    seller_content = content.find("tbody")

                    if seller_content is not None:
                        seller_content = seller_content.findAll("td")[0]
                        
                        if seller_content is not None:
                            seller_content = seller_content.find("a")
                            
                            if seller_content is not None:
                                seller = get_content(seller_content)

                    if description != "" or title != "":
#                         # Insert product into database
                        query = "INSERT INTO products_tmp_anarchia (name, description, seller_name, market_name, has_image, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                        values = (title, description, seller, "Anarchia", "no_image", "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid


                        ## GET IMAGE
                        raw_data = page_content.find("div", attrs={"id": "images"})

                        if raw_data is not None:
                            raw_data = raw_data.find("a")
                            
                            if raw_data is not None:
                                img_path = raw_data["href"]
                                name = img_path.replace("http://tkmlejimlt72jnkm.onion/index.php/", "")

                                if name is not "":
                                    url_image = Path(f"{folder_image}\index.php\{name}")
                                    
                                    print(f"{product_id} - {url_image}")

#                                     # copy images to new directory, renaming with product id
#                                     if url_image.exists():
#                                         print(f"Image {product_id} exists... creating")
#                                         dst_dir = Path(f"D:\images\_anarchia\{product_id}.jpeg")

#                                         with open(url_image) as img_file:
#                                             data = img_file.read()
#                                             imgc_data = "" + data.replace("data:image/jpeg;base64,", "")
#                                             img_data = base64.b64decode(imgc_data)

#                                             with open(dst_dir, 'wb') as f:
#                                                 f.write(img_data)
#                                                 f.close
#                                             img_file.close
# #                                             copyfile(url_image, dst_dir)
# #                                             update_image(product_id, "has_image")
#                                     else:
#                                         print(f"Image {product_id} does not exists")
#                                 else:
#                                     print(f"Image {product_id} has no name")